# AutoML with H20.ai
This notebook will be a part of Modeling node of the Klee project. This is a Jupyter Notebook we will execute the autoML using H20.ai library. This is part of the **Modeling** component of the Klee project.

- H2O’s AutoML can be used for automating the machine learning workflow, which includes automatic training and tuning of many models within a user-specified time-limit.
- H2O’s AutoML can also be a helpful tool for the advanced user, by providing a simple wrapper function that performs a large number of modeling-related tasks that would typically require many lines of code, and by freeing up their time to focus on other aspects of the data science pipeline tasks such as data-preprocessing, feature engineering and model deployment.

##### We can see in this notebook that the ensemble model from h20.ai works best, as it is a combination of all the models tested.

##### Limitation:
The h20.ai library tests on alot of models, but because of low computational power on my system, it is testing on mostly tree based models. 

### Libraries used 
- We use H20.AI to investigate and select best features from the dataset. 

#### Input: 
The input to this notebook is Tabular dataset. Ex. Titanic dataset used in this notebook

#### Output:
Output of this notebook is the best model based on performance accuracy for the dataset

In [1]:
# import the library

import h2o
from h2o.automl import H2OAutoML
import os
import pandas as pd

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 25.221-b11, mixed mode)
  Starting server from C:\Users\Abhishek\Anaconda3\lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Abhishek\AppData\Local\Temp\tmp41d858y9
  JVM stdout: C:\Users\Abhishek\AppData\Local\Temp\tmp41d858y9\h2o_Abhishek_started_from_python.out
  JVM stderr: C:\Users\Abhishek\AppData\Local\Temp\tmp41d858y9\h2o_Abhishek_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,America/New_York
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.3
H2O_cluster_version_age:,11 days
H2O_cluster_name:,H2O_from_python_Abhishek_0glhyh
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.530 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


# Load the dataset

In [3]:
# Use local data file or download from GitHub

data_path = "test.csv"

pd.read_csv(data_path)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [8]:
# load data into h20 

df = h2o.import_file(data_path)
# df = pd.read_csv(data_path)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [9]:
# descriptive statistics for the dataset

df.describe()

Rows:418
Cols:11




,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
type,int,int,string,enum,real,int,int,int,real,enum,enum
mins,892.0,1.0,NaN,,0.17,0.0,0.0,680.0,0.0,,
mean,1100.5,2.2655502392344493,NaN,,30.272590361445786,0.44736842105263147,0.39234449760765555,223850.98986486494,35.627188489208656,,
maxs,1309.0,3.0,NaN,,76.0,8.0,9.0,3101298.0,512.3292,,
sigma,120.81045760473994,0.8418375519640504,NaN,,14.181209235624422,0.8967595611217135,0.9814288785371694,369523.7764694362,55.907576179973844,,
zeros,0,0,0,,0,283,324,0,2,,
missing,0,0,0,0,86,0,0,122,1,327,0
0,892.0,3.0,"Kelly, Mr. James",male,34.5,0.0,0.0,330911.0,7.8292,,Q
1,893.0,3.0,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1.0,0.0,363272.0,7.0,,S
2,894.0,2.0,"Myles, Mr. Thomas Francis",male,62.0,0.0,0.0,240276.0,9.6875,,Q


In [10]:
# define the independent and dependent variable
y = "Fare"

In [11]:
# split the data into train and test split

splits = df.split_frame(ratios = [0.8], seed = 1)
train = splits[0]
test = splits[1]

In [12]:
splits

PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,,Q
893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47,1,0,363272,7,,S
895,3,"Wirz, Mr. Albert",male,27,0,0,315154,8.6625,,S
896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22,1,1,3.1013e+06,12.2875,,S
897,3,"Svensson, Mr. Johan Cervin",male,14,0,0,7538,9.225,,S
898,3,"Connolly, Miss. Kate",female,30,0,0,330972,7.6292,,Q
899,2,"Caldwell, Mr. Albert Francis",male,26,1,1,248738,29,,S
900,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18,0,0,2657,7.2292,,C
902,3,"Ilieff, Mr. Ylio",male,nan,0,0,349220,7.8958,,S
903,1,"Jones, Mr. Charles Cresson",male,46,0,0,694,26,,S


PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
894,2,"Myles, Mr. Thomas Francis",male,62,0,0,240276,9.6875,,Q
901,3,"Davies, Mr. John Samuel",male,21,2,0,nan,24.15,,S
912,1,"Rothschild, Mr. Martin",male,55,1,0,nan,59.4,,C
929,3,"Cacic, Miss. Manda",female,21,0,0,315087,8.6625,,S
931,3,"Hee, Mr. Ling",male,nan,0,0,1601,56.4958,,S
943,2,"Pulbaum, Mr. Franz",male,27,0,0,nan,15.0333,,C
947,3,"Rice, Master. Albert",male,10,4,1,382652,29.125,,Q
955,3,"Bradley, Miss. Bridget Delia",female,22,0,0,334914,7.725,,Q
956,1,"Ryerson, Master. John Borie",male,13,2,2,nan,262.375,B57 B59 B63 B66,C
964,3,"Nieminen, Miss. Manta Josefina",female,29,0,0,3.1013e+06,7.925,,S


[, ]

# AutoML
- Run AutoML, stopping after 60 seconds. The **max_runtime_secs** argument provides a way to limit the AutoML run by time. When using a time-limited stopping criterion, the number of models train will vary between runs. If different hardware is used or even if the same machine is used but the available compute resources on that machine are not the same between runs, then AutoML may be able to train more models on one run vs another.

- The test frame is passed explicitly to the leaderboard_frame argument here, which means that instead of using cross-validated metrics, we use test set metrics for generating the leaderboard.

In [ ]:
aml = H2OAutoML(max_runtime_secs = 60, seed = 1, project_name = "Testing_frame")
aml.train(y = y, training_frame = train, leaderboard_frame = test)

AutoML progress: |
18:11:00.236: AutoML: XGBoost is not available; skipping it.
18:11:00.253: Step 'best_of_family_xgboost' not defined in provider 'StackedEnsemble': skipping it.
18:11:00.253: Step 'all_xgboost' not defined in provider 'StackedEnsemble': skipping it.
11:37:36.907: New models will be added to existing leaderboard Testing_frame@@Fare (leaderboard frame=py_3_sid_a3a4) with already 49 models.
11:37:36.907: AutoML: XGBoost is not available; skipping it.
11:37:36.907: Step 'best_of_family_xgboost' not defined in provider 'StackedEnsemble': skipping it.
11:37:36.907: Step 'all_xgboost' not defined in provider 'StackedEnsemble': skipping it.

███████
11:37:43.990: StackedEnsemble_BestOfFamily_5_AutoML_2_20211014_113736 [StackedEnsemble best_of_family_2 (built with AUTO metalearner, using top model from each algorithm type)] failed: water.exceptions.H2OIllegalArgumentException: Failed to find the xval predictions frame. . .  Looks like keep_cross_validation_predictions wasn't 

## Leaderboard
- View the AutoML Leaderboard. Since we specified a leaderboard_frame in the H2OAutoML.train() method for scoring and ranking the models, the AutoML leaderboard uses the performance on this data to rank the models.

- A default performance metric for each machine learning task (binary classification, multiclass classification, regression) is specified internally and the leaderboard will be sorted by that metric. In the case of regression, the default ranking metric is mean residual deviance. In the future, the user will be able to specify any of the H2O metrics so that different metrics can be used to generate rankings on the leaderboard.

In [22]:
aml.leaderboard.head()

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_BestOfFamily_2_AutoML_1_20211013_181100,937.551,30.6195,937.551,16.5652,0.990622
GBM_grid_1_AutoML_1_20211013_181100_model_9,978.155,31.2755,978.155,15.1538,0.505271
StackedEnsemble_AllModels_1_AutoML_1_20211013_181100,981.505,31.329,981.505,16.7283,0.87147
StackedEnsemble_BestOfFamily_3_AutoML_1_20211013_181100,995.181,31.5465,995.181,15.5825,0.612669
GBM_grid_1_AutoML_1_20211013_181100_model_10,1019.92,31.9362,1019.92,14.6545,0.537303
GBM_grid_1_AutoML_1_20211013_181100_model_17,1020.91,31.9516,1020.91,14.5714,0.515761
GBM_2_AutoML_1_20211013_181100,1026.35,32.0368,1026.35,13.3962,0.487166
StackedEnsemble_AllModels_2_AutoML_1_20211013_181100,1031.83,32.1221,1031.83,15.6841,0.608232
GBM_grid_1_AutoML_1_20211013_181100_model_2,1032.56,32.1335,1032.56,14.2254,0.492624
GBM_5_AutoML_1_20211013_181100,1064.86,32.6321,1064.86,14.2371,0.484867


Mostly on the top there will be ensemble of models, as they perform better than a single model

## Testing of model

In [23]:
perf = aml.leader.model_performance(test)
perf


ModelMetricsRegressionGLM: stackedensemble
** Reported on test data. **

MSE: 937.5513364583173
RMSE: 30.619460094167522
MAE: 16.56518781190623
RMSLE: 0.9906215169332465
R^2: 0.6755365000824696
Mean Residual Deviance: 937.5513364583173
Null degrees of freedom: 83
Residual degrees of freedom: 80
Null deviance: 242885.75655845192
Residual deviance: 78754.31226249866
AIC: 823.2164808580599


In [24]:
# Get model ids for all models in the AutoML Leaderboard
model_ids = list(aml.leaderboard['model_id'].as_data_frame().iloc[:,0])
# Get the "All Models" Stacked Ensemble model
se = h2o.get_model([mid for mid in model_ids if "StackedEnsemble_AllModels" in mid][0])
# Get the Stacked Ensemble metalearner model
metalearner = se.metalearner()

In [25]:
metalearner.coef_norm()

{'Intercept': 35.35996883798829,
 'GBM_2_AutoML_1_20211013_181100': 1.6771024065601614,
 'GBM_4_AutoML_1_20211013_181100': 28.387593613334428,
 'GBM_3_AutoML_1_20211013_181100': 0.0015506532170065127,
 'DRF_1_AutoML_1_20211013_181100': 0.0,
 'GLM_1_AutoML_1_20211013_181100': 13.7485312905518,
 'GBM_1_AutoML_1_20211013_181100': 0.0}

## Save the best Model

In [27]:
h2o.save_model(aml.leader, path = "./BestModel")

'C:\\Users\\Abhishek\\Desktop\\Kinesso\\BestModel\\StackedEnsemble_BestOfFamily_2_AutoML_1_20211013_181100'